Notebook pour pousser les données dans BigQuery en local.

In [2]:
import pandas as pd
from pandas_gbq import to_gbq
import json
from pathlib import Path
from google.oauth2 import service_account

path = Path.cwd().parent

# Get bq authentication
with open(path / "secrets/dash-vlille-95a18802a3a1.json") as f:
    credentials = json.load(f)

# Get table path
with open(path / "secrets/data_access.json") as f:
    config = json.load(f)
project_id = config["GCP_PROJECT_ID"]
dataset_id = config["BIGQUERY_DATASET_ID"]
table_id = config["BIGQUERY_TABLE_ID"]
table_full_id = f"{project_id}.{dataset_id}.{table_id}"

key_path = path / "secrets/dash-vlille-95a18802a3a1.json"
credentials = service_account.Credentials.from_service_account_file(key_path)

In [ ]:
df_columns = pd.read_csv(
    path / "data/vlille_data_01_11_2025_08_55.csv", nrows=1
).columns.tolist()
df_columns.append("Date_Scrapping")
df_columns.append("Heure_Min_Scrapping")

In [ ]:
# Load data
df = pd.read_csv(path / "Data_frame/Joined_DF_March.csv", header=None)
df.columns = df_columns
df.head()

,type,@typeName,@id,geometry.type,geometry.@name,geometry.@srs,geometry.coordinates,properties.nom,properties.adresse,properties.code_insee,...,properties.etat,properties.type,properties.nb_places_dispo,properties.nb_velos_dispo,properties.etat_connexion,properties.x,properties.y,properties.date_modification,Date_Scrapping,Heure_Min_Scrapping
0,Feature,ilevia:vlille_temps_reel,vlille_temps_reel.33,Point,geom,EPSG:4326,"[3.035681, 50.63022]",PORT DE LILLE,Rue de Lille (métro Port de Lille),NaN,...,EN SERVICE,AVEC TPE,15,11,CONNECTÉ,3.035681,50.630220,2025-03-01T02:39:08.205+00:00,01_03_2025,01_40.c
1,Feature,ilevia:vlille_temps_reel,vlille_temps_reel.40,Point,geom,EPSG:4326,"[3.051729, 50.62614]",METRO GAMBETTA,43 rue du Marché,NaN,...,EN SERVICE,AVEC TPE,16,12,CONNECTÉ,3.051729,50.626140,2025-03-01T02:39:08.207+00:00,01_03_2025,01_40.c
2,Feature,ilevia:vlille_temps_reel,vlille_temps_reel.45,Point,geom,EPSG:4326,"[3.05659, 50.62633]",BRULE MAISON,61 Rue des Postes,NaN,...,EN SERVICE,AVEC TPE,17,1,CONNECTÉ,3.056590,50.626330,2025-03-01T02:39:08.210+00:00,01_03_2025,01_40.c
3,Feature,ilevia:vlille_temps_reel,vlille_temps_reel.51,Point,geom,EPSG:4326,"[3.068689, 50.62482]",JB LEBAS,76 boulevard Jean-Baptiste Lebas,NaN,...,EN SERVICE,AVEC TPE,15,25,CONNECTÉ,3.068689,50.624820,2025-03-01T02:39:08.212+00:00,01_03_2025,01_40.c
4,Feature,ilevia:vlille_temps_reel,vlille_temps_reel.56,Point,geom,EPSG:4326,"[3.055471, 50.620304]",BD VICTOR HUGO,216 Boulevard Victor Hugo,NaN,...,EN SERVICE,AVEC TPE,9,9,CONNECTÉ,3.055471,50.620304,2025-03-01T02:39:08.213+00:00,01_03_2025,01_40.c


In [ ]:
drop_columns = [
    "type",
    "@typeName",
    "geometry.type",
    "geometry.@name",
    "geometry.@srs",
    "properties.code_insee",
    "properties.type",
]
df.drop(columns=drop_columns, inplace=True)
df.rename(columns={"@id": "ID_station"}, inplace=True)
df.columns = df.columns.str.replace(".", "_")


In [11]:
# Upload data to BigQuery
to_gbq(
    df,
    table_full_id,
    project_id=project_id,
    if_exists="append",
    credentials=credentials,
)


Adaptation du script scrapper pour pusher directement dans BigQuery via GitHub Actions.

In [ ]:
import json
import requests
import pandas as pd
from datetime import datetime


def call_data():
    api_url = "https://data.lillemetropole.fr/data/ogcapi/collections/ilevia:vlille_temps_reel/items?f=geojson&limit=-1"
    api_call = requests.get(api_url)
    api_data = api_call.text
    api_data = json.loads(api_data)
    df = [feature for feature in api_data["features"]]
    df = pd.json_normalize(df)
    return df


if __name__ == "__main__":
    df = call_data()
    date = datetime.now().strftime("%d_%m_%Y_%H_%M")
    drop_columns = [
        "type",
        "@typeName",
        "geometry.type",
        "geometry.@name",
        "geometry.@srs",
        "properties.code_insee",
        "properties.type",
    ]
    df.drop(columns=drop_columns, inplace=True)
    df.rename(columns={"@id": "ID_station"}, inplace=True)
    df.columns = df.columns.str.replace(".", "_")

    df["Date_Scrapping"] = datetime.now().strftime("%d/%m/%Y")
    df["Heure_Min_Scrapping"] = datetime.now().strftime("%H:%M")
    to_gbq(
        df,
        table_full_id,
        project_id=project_id,
        if_exists="append",
        credentials=credentials,
    )


,ID_station,geometry_coordinates,properties_nom,properties_adresse,properties_commune,properties_etat,properties_nb_places_dispo,properties_nb_velos_dispo,properties_etat_connexion,properties_x,properties_y,properties_date_modification,Date_Scrapping,Heure_Min_Scrapping
0,vlille_temps_reel.7,"[3.055307, 50.633728]",PLACE DE STRASBOURG,7 Place de Strasbourg,Lille,EN SERVICE,12,4,CONNECTÉ,3.055307,50.633728,2025-11-02T00:27:08.415+00:00,02/11/2025,00:28
1,vlille_temps_reel.11,"[3.060977, 50.63734]",NOUVEAU SIECLE,16 rue de Pas,Lille,EN SERVICE,20,12,CONNECTÉ,3.060977,50.637340,2025-11-02T00:27:08.417+00:00,02/11/2025,00:28
2,vlille_temps_reel.70,"[3.073991, 50.63424]",CITE ADMINISTRATIVE,51 rue de Tournai,Lille,EN SERVICE,15,1,CONNECTÉ,3.073991,50.634240,2025-11-02T00:27:08.443+00:00,02/11/2025,00:28
3,vlille_temps_reel.71,"[3.030778, 50.634045]",BOIS BLANCS,124 Avenue de Dunkerque,Lille,EN SERVICE,21,1,CONNECTÉ,3.030778,50.634045,2025-11-02T00:27:08.443+00:00,02/11/2025,00:28
4,vlille_temps_reel.186,"[3.092388, 50.672455]",MARCQ MAIRIE,1 Avenue du Maréchal De Lattre De Tassigny,Marcq en Barœul,EN SERVICE,3,8,CONNECTÉ,3.092388,50.672455,2025-11-02T00:27:08.480+00:00,02/11/2025,00:28
